In [1]:
import numpy as np

In [373]:
# k: number of classes
# labels: a mxn matrix, labels of n items by m workers
# Z: a 3xnxk matrix, the group aggregated labels in frequency
def get_Z1(k,labels,groups):
    m,n = labels.shape
    Z = np.zeros((3,n,k))
    for i in range(m):
        for j in range(n):
            if labels[i,j]<k:
                Z[groups[i],j,labels[i,j]]+=1.
    for g in range(3):
        Z[g,:,:]/=np.sum(groups==g)
    return Z


In [391]:
def get_Zg(k, labels, groups):
    m, n = labels.shape
    Zg = np.zeros((3, n, k))
    for g in range(3):
        for l in range(k):
            Zg[g, :, l] = np.mean(labels[groups == g, :] == l, axis=0)
    return Zg


In [162]:
k = 10
labels = np.random.randint(k,size=(100,1000))

In [185]:
Z1 = get_Z1(k, labels)

In [183]:
Z2 = get_Z2(k, labels)

In [186]:
np.all(Z1==Z2)

True

In [148]:
Z1

array([[[0.17948718, 0.02564103, 0.1025641 , ..., 0.1025641 ,
         0.02564103, 0.02564103],
        [0.12820513, 0.05128205, 0.07692308, ..., 0.15384615,
         0.12820513, 0.07692308],
        [0.07692308, 0.12820513, 0.07692308, ..., 0.12820513,
         0.02564103, 0.1025641 ],
        ...,
        [0.12820513, 0.02564103, 0.05128205, ..., 0.07692308,
         0.17948718, 0.02564103],
        [0.07692308, 0.15384615, 0.05128205, ..., 0.1025641 ,
         0.        , 0.15384615],
        [0.15384615, 0.1025641 , 0.07692308, ..., 0.07692308,
         0.07692308, 0.15384615]],

       [[0.05882353, 0.14705882, 0.05882353, ..., 0.08823529,
         0.11764706, 0.08823529],
        [0.11764706, 0.11764706, 0.08823529, ..., 0.05882353,
         0.17647059, 0.08823529],
        [0.08823529, 0.17647059, 0.05882353, ..., 0.08823529,
         0.08823529, 0.02941176],
        ...,
        [0.05882353, 0.02941176, 0.20588235, ..., 0.08823529,
         0.14705882, 0.17647059],
        [0.1

In [56]:
Z1.shape

(3, 1000, 10)

get_Z2 is more efficient.

In [57]:
get_Z = get_Z2

In [65]:
Z = Z2

In [67]:
a,b,c = 1,2,0

In [94]:
Z1 = Z[0,:,:]
Z2 = Z[1,:,:]


In [390]:
def get_M(Zg):
    _, n, k = Zg.shape
    M2s, M3s = [], []
    for a, b, c in [(1, 2, 0), (2, 0, 1), (0, 1, 2)]:
        Za = (Zg[c, :, :].T@Zg[b, :, :]/n)@np.linalg.inv(Zg[a, :, :].T @
                                                         Zg[b, :, :]/n)@Zg[a, :, :].T
        Zb = (Zg[c, :, :].T@Zg[a, :, :]/n)@np.linalg.inv(Zg[b, :, :].T @
                                                         Zg[a, :, :]/n)@Zg[b, :, :].T
        M2s.append(Za@Zb.T/n)
        M3s.append(np.einsum("ji,ki,li->jkl", Za, Zb, Zg[c, :, :].T)/n)
    return M2s, M3s


In [117]:
n = 1000
k = 10
m = 100


In [124]:
Za = ((Z[c, :, :].T@Z[b, :, :]/n)@np.linalg.inv(Z[a, :, :].T @
                                                Z[b, :, :]/n)@Z[a, :, :].T).T

Zb = ((Z[c, :, :].T@Z[a, :, :]/n)@np.linalg.inv(Z[b, :, :].T @
                                               Z[a, :, :]/n)@Z[b, :, :].T).T


In [125]:
Za.shape

(1000, 10)

In [120]:
Zc = Z[c, :, :]


In [128]:
Zc.shape

(1000, 10)

In [138]:
np.einsum("ij,ik,il->jkl", Za, Zb, Zc)[4,2,3]


1.1818491928598645

In [139]:
np.sum(Za[:,4]*Zb[:,2]*Zc[:,3])

1.181849192859866

In [211]:
M2s,M3s = get_M(Z)

In [380]:
def get_whiten(M2, sym=True):
    # closest symmetric matrix
    if sym:
        M2 = (M2+M2.T)/2
    u, s, vh = np.linalg.svd(M2, full_matrices=False)
    return u@np.diag(s**-0.5)


In [260]:
Q = get_whiten(M2)


In [259]:
def whiten_tensor(M3, Q):
    return np.einsum("def,da,eb,fc->abc", M3, Q, Q, Q)


In [263]:
M3_whiten = whiten_tensor(M3, Q)


In [230]:
M3 = M3s[1]

In [235]:
res2 = np.einsum("def,da,eb,fc->abc", M3, Q, Q, Q)


In [243]:
res = 0
for d in range(k):
    for e in range(k):
        for f in range(k):
            res += M3[d, e, f]*Q[d, 3]*Q[e, 6]*Q[f, 4]
res

-6.7722753191269955

In [251]:
np.einsum("de,da,eb->ab", M2, Q, Q)


array([[ 1.00000000e+00,  2.92724274e-04, -7.70266597e-04,
        -3.24706358e-04,  9.34030185e-05, -6.82135895e-04,
        -1.87419387e-04,  3.25637463e-04, -2.01896539e-04,
         7.74038037e-04],
       [-2.92724274e-04,  1.00000000e+00, -7.15399702e-01,
        -1.90536688e-01, -1.55590714e-01, -2.00915229e-01,
        -1.38560236e-01,  1.70875810e-01, -2.80267574e-01,
         7.65161835e-01],
       [ 7.70266597e-04,  7.15399702e-01, -1.00000000e+00,
         5.71519242e-01,  5.15232004e-01, -4.07964125e-02,
        -9.66429912e-02, -4.28606367e-01, -3.73505574e-01,
        -3.54732621e-01],
       [ 3.24706358e-04,  1.90536688e-01, -5.71519242e-01,
         1.00000000e+00,  3.53937874e-01,  5.07659393e-01,
        -1.76639189e-01, -8.70426302e-02, -1.22783086e-01,
         3.73887298e-02],
       [-9.34030185e-05,  1.55590714e-01, -5.15232004e-01,
        -3.53937874e-01,  1.00000000e+00, -7.85781006e-01,
         3.15481561e-02, -2.19125095e-01,  4.73015660e-01,
         2.

In [253]:
np.allclose(np.einsum("de,da,eb->ab", M2, Q, Q),Q.T@M2@Q)


True

In [255]:
whiten_tensor(M3s, Qs)

[array([[[-9.99999735e-01, -3.12669940e-04, -7.73047154e-05,
           5.67358391e-05,  1.27543869e-04, -7.01031565e-06,
           2.81130119e-04, -1.47187278e-04, -1.48509206e-04,
           7.34790316e-05],
         [-2.97909831e-04,  9.97880292e-01, -1.09867530e+00,
           1.06354001e-01,  5.92418695e-01,  4.35360696e-01,
          -2.79298060e-01, -1.62750474e-01,  2.87028966e-01,
          -3.54840859e-01],
         [-3.89096676e-04,  1.09531494e+00, -1.00061598e+00,
          -3.34145221e-01, -7.33585698e-02, -4.15720321e-01,
           3.05227879e-01, -4.27848772e-01,  2.18976465e-01,
           1.03194184e-02],
         [ 1.45890957e-04, -1.07953170e-01,  3.33679895e-01,
          -9.99147960e-01, -8.59312563e-01, -3.85624341e-01,
           7.21826994e-01,  2.87042486e-01, -4.37742223e-01,
          -6.31455694e-03],
         [ 3.05077997e-04, -5.96563505e-01,  6.88865406e-02,
           8.60825402e-01,  1.00249481e+00,  3.98680595e-01,
           9.46529881e-02, -4.0242

In [278]:
def sym_tensor(T):
    return (T+T.transpose(0, 2, 1)+T.transpose(1, 0, 2)+T.transpose(1, 2, 0)+T.transpose(2, 0, 1)+T.transpose(2, 1, 0))/6

In [280]:
T = sym_tensor(M3_whiten)


In [357]:
def robust_tensor_power(T,L=20,N=100,sym=True):
    k,_,_ = T.shape
    if sym:
        T = sym_tensor(T)
    values = np.zeros(k)
    vectors = np.zeros((k,k))
    for h in range(k):
        theta_tau = []
        for tau in range(L):
            theta = np.random.randn(k)
            theta /= np.linalg.norm(theta)
            for t in range(N):
                theta = np.einsum("aef,e,f->a", T, theta, theta)
                theta /= np.linalg.norm(theta)
            theta_tau.append(theta)

        lam_best = float("-Inf")
        theta_best = np.array([])
        for theta in theta_tau:
            lam = np.einsum("efg,e,f,g->", T, theta, theta, theta)
            if lam > lam_best:
                lam_best = lam
                theta_best = theta

        theta = theta_best
        for t in range(N):
            theta = np.einsum("aef,e,f->a", T, theta, theta)
            theta /= np.linalg.norm(theta)
        lam = np.einsum("efg,e,f,g->", T, theta, theta, theta)

        values[h] = lam
        vectors[:,h] = theta
        T = T-lam*np.einsum("a,b,c->abc",theta,theta,theta)
    return values,vectors
        


In [317]:
robust_tensor_power(M3_whiten, L=10, N=10, sym=True)


(array([55.86007863, 46.28781448, 32.02288236, 28.03589382, 24.00997777,
        17.19309568, 20.55047251, 17.60298255, 16.36828785, 14.62381024]),
 array([[-0.0089957 , -0.00625538, -0.00299219, -0.00854321,  0.02476507,
         -0.01011561,  0.02244343,  0.01442269, -0.01156807,  0.01623874],
        [-0.02384345, -0.43883026,  0.58969942, -0.71764074, -0.11804595,
          0.57576307,  0.10830866, -0.07431096, -0.47333744, -0.2131938 ],
        [ 0.21978952,  0.10621439, -0.20538141,  0.09857403, -0.65714189,
          0.05139225, -0.37193204, -0.42977841,  0.11376737,  0.61382457],
        [ 0.06046089, -0.27549765, -0.15599577,  0.12510165,  0.23028582,
          0.31340743, -0.366232  , -0.17700086, -0.29260798,  0.32959026],
        [ 0.32665591, -0.30166724,  0.26856112, -0.18901346, -0.05371045,
          0.24288718, -0.38657396, -0.0988299 ,  0.04408979, -0.41789853],
        [ 0.19286258, -0.4050995 , -0.23931377, -0.04237925,  0.09286482,
          0.09723358, -0.48031135

In [292]:
theta = np.random.randn(k)
theta /= np.linalg.norm(theta)


In [290]:
theta[0]**2+theta[1]**2+theta[2]**2


0.9999999999999999

In [293]:
np.einsum("def,da,e,f->a", M3, np.eye(k), theta, theta)


array([0.00507942, 0.00547478, 0.00534615, 0.00542853, 0.00541378,
       0.00524568, 0.0052369 , 0.00536064, 0.00534583, 0.00537816])

In [294]:
np.einsum("aef,e,f->a", M3, theta, theta)


array([0.00507942, 0.00547478, 0.00534615, 0.00542853, 0.00541378,
       0.00524568, 0.0052369 , 0.00536064, 0.00534583, 0.00537816])

In [300]:
np.einsum("efg,e,f,g->", M3, theta, theta, theta)

0.01230363368709415

In [301]:
float("-Inf")


-inf

In [303]:
np.array([])


array([], dtype=float64)

In [307]:
theta[0]**2*theta[2]

0.0020785810846164813

In [304]:
np.einsum("a,b,c->abc", theta, theta, theta)


array([[[ 3.88369478e-03,  1.16414860e-02,  2.07858108e-03,
          9.22695984e-03,  1.98588148e-04,  4.63339141e-03,
          1.07777416e-02, -5.31326091e-03,  7.70975810e-03,
          1.20441248e-02],
        [ 1.16414860e-02,  3.48956867e-02,  6.23060615e-03,
          2.76580756e-02,  5.95273645e-04,  1.38887231e-02,
          3.23065882e-02, -1.59266513e-02,  2.31102201e-02,
          3.61026080e-02],
        [ 2.07858108e-03,  6.23060615e-03,  1.11247139e-03,
          4.93833457e-03,  1.06285790e-04,  2.47982406e-03,
          5.76832400e-03, -2.84369505e-03,  4.12631740e-03,
          6.44610131e-03],
        [ 9.22695984e-03,  2.76580756e-02,  4.93833457e-03,
          2.19215960e-02,  4.71809700e-04,  1.10081041e-02,
          2.56059743e-02, -1.26233517e-02,  1.83169977e-02,
          2.86146729e-02],
        [ 1.98588148e-04,  5.95273645e-04,  1.06285790e-04,
          4.71809700e-04,  1.01545706e-05,  2.36923001e-04,
          5.51107095e-04, -2.71687325e-04,  3.942293

In [315]:
T

array([[[-1.00000210e+00, -1.18980970e-03,  4.37587141e-04,
          3.88472950e-04, -3.51689446e-04,  3.59012996e-04,
          2.01515403e-04, -1.20271063e-04,  9.98559142e-05,
         -9.83184087e-05],
        [-1.18980970e-03, -1.00795174e+00,  3.97078988e-03,
         -3.77379100e-03,  3.83140463e-03, -3.99473157e-03,
          6.18616835e-03, -1.36095470e-02, -6.08876169e-03,
          1.66932148e-02],
        [ 4.37587141e-04,  3.97078988e-03,  9.94263932e-01,
          2.42335164e-03,  3.01801192e-03,  3.95131031e-03,
         -3.35000194e-03, -1.22834693e-03, -2.54529811e-03,
         -6.50428638e-03],
        [ 3.88472950e-04, -3.77379100e-03,  2.42335164e-03,
         -9.95569584e-01, -1.99096800e-03, -1.11502534e-03,
          8.64940177e-03,  7.01177571e-03,  4.64597936e-03,
         -3.57985913e-03],
        [-3.51689446e-04,  3.83140463e-03,  3.01801192e-03,
         -1.99096800e-03, -1.00006038e+00, -7.26963197e-03,
         -2.08850236e-03,  1.01806234e-03,  1.093108

In [392]:
def get_confusion_matrix(k, labels, groups=None, sym=True, L=20, N=100, seed=0):
    m, n = labels.shape
    if not groups:
        np.random.seed(seed)
        groups = np.random.randint(3, size=m)
    Zg = get_Zg(k, labels, groups)
    M2s, M3s = get_M(Zg)
    Cc = np.zeros((3,k,k))
    W = np.zeros((3,k,k))
    for g, (M2, M3) in enumerate(zip(M2s, M3s)):
        Q = get_whiten(M2, sym)
        M3_whiten = whiten_tensor(M3, Q)
        values, vectors = robust_tensor_power(M3_whiten, L, N, sym)
        w = values**-2
        mu = np.linalg.inv(Q.T)@vectors@np.diag(values)
        best = np.argmax(mu,axis = 1)
        for h in range(k):
            Cc[g,:,h] = mu[:,best[h]]
            W[g,h,h] = w[h]
    W = np.mean(W,axis=0)
    C = np.zeros((m,k,k))
    for i in range(m):
        Ca = (np.sum(Cc,axis=0)-Cc[groups[i],:,:])/2
        Za = (np.sum(Zg, axis=0)-Zg[groups[i], :, :])/2
        E = np.zeros((k,k))
        for j in range(n):
            E[labels[i,j],:] += Za[j,:]
        E/=N
        Ci = E@np.linalg.inv(W@Ca.T)
        for l in range(k):
            Ci[:, l] /= np.linalg.norm(Ci[:, l])
        C[i,:,:] = Ci
    return C
    


In [394]:
get_confusion_matrix(k, labels, groups=None, sym=True, L=20, N=100, seed=0).shape


(100, 10, 10)

In [358]:
values, vectors = robust_tensor_power(M3_whiten,L=20,N=100)


In [366]:
mu = np.linalg.inv(Q.T)@vectors@np.diag(values)


In [371]:
mu

array([[ 2.51936743e-01, -5.78802447e-01,  2.22557411e-01,
        -3.37064754e-01, -1.44107775e-01, -1.54436447e-01,
         2.42191443e-01,  4.54392896e-02, -5.29886474e-02,
        -1.35084346e-01],
       [ 2.09234089e-01, -4.36581761e-02, -8.43550717e-02,
        -1.46976320e-01, -6.24549476e-02,  8.19100085e-02,
         5.45680380e-02, -4.98903548e-02,  5.88009177e-02,
        -5.79751933e-04],
       [-3.31400624e-01, -2.73197316e-01,  6.29531422e-01,
        -5.85573389e-01,  5.35475726e-02,  3.11164730e-01,
         7.36099381e-02,  8.31026295e-02, -2.21749472e-01,
        -2.14775118e-01],
       [-4.94088070e-02,  2.87033459e-01, -3.04940139e-01,
         5.20741147e-01,  1.89506838e-01, -1.63832732e-01,
        -1.31075824e-01,  4.86073995e-02,  2.29435244e-01,
        -7.91712397e-02],
       [ 4.01665218e-01,  2.13980192e-01, -2.20286515e-01,
         2.11973733e-01, -2.06765617e-01, -2.05540711e-01,
         8.09690481e-02, -1.31142979e-01,  9.54553091e-02,
         5.

In [369]:
np.argmax(mu)


71

In [384]:
for g, (M2, M3) in enumerate(zip(M2s, M3s)):
    print(g,M2,M3)

0 [[0.00998153 0.00983773 0.01000958 0.01025642 0.01016982 0.0097797
  0.00983088 0.0099041  0.00967342 0.00971064]
 [0.00962651 0.00983881 0.00957723 0.0101639  0.00987138 0.01008239
  0.01022152 0.00969766 0.00985165 0.0100946 ]
 [0.00962001 0.00981565 0.00960386 0.01008069 0.00962317 0.01030468
  0.01053639 0.00952553 0.01009086 0.01023506]
 [0.01010867 0.01027931 0.01003575 0.01040898 0.01020267 0.01058957
  0.01063287 0.00995237 0.01046188 0.0103792 ]
 [0.01016301 0.00995979 0.01038034 0.01061413 0.01042888 0.0098975
  0.00998818 0.01013823 0.00983576 0.00985059]
 [0.00972542 0.01012615 0.00952888 0.01027866 0.0103321  0.01004279
  0.00995495 0.01007229 0.00963095 0.01025653]
 [0.01021681 0.00998655 0.00993373 0.01034711 0.01023217 0.01008525
  0.0103151  0.01007734 0.00978727 0.01017252]
 [0.00992052 0.00977277 0.00995921 0.01012099 0.01005999 0.009823
  0.00991816 0.00975501 0.00981975 0.00964548]
 [0.00981215 0.00974665 0.00984419 0.01018918 0.00997676 0.00989409
  0.00996478 0